# Applied Artificial Intelligence - Lab 2

Luca van Straaten - 18073611

## Preparation

First I uninstalled anaconda by:
- removing the lines from my .config/fish/config.fish
- removing the path from fish
- `brew uninstall anaconda`

Than I installed miniconda by:

```fish
brew install miniconda
```

That was easy.

uh, it did not install the newest version of conda, so I had to update it by:

```fish
conda update -n base -c defaults conda
```

**Maby make a pull request to the brew formula? - [if I ever feel like it]™**

And I recreated the environment with python 3.10:

```fish
conda create -n aai_lab python=3.10
conda activate aai_lab
conda install tensorflow notebook pandas matplotlib numpy
conda install scikit-learn
conda install -c conda-forge nb_conda_kernels
```

**System information**: 2018 Intel Core i7 13-inch MacBookPro15,2, 16GB RAM, 512GB SSD, macOS Ventura 13.0 (22A380), kernel 22.1.0

This file, along with the rest of the labs, are tracked in a git repository on github. [lab 2](https://github.com/lucanatorvs/Applied_Artificial_Intelligence_Lab/blob/main/2/lab2.ipynb)

## Exercise 1 - A quick overview of machine learning project.

The end goal of this lab is to train a linear regression model to make an insurance cost prediction. Write the necessary code and answer the questions below.

### 1

Use the CSV file from Blackboard Med_insurance.csv [2] from last week. It contains data of medical information and insurance cost. It contains 1338 rows of data with columns: age, gender, BMI, children, smoker, region, insurance charges. Read this csv file using pandas library into a variable called insurance_data

In [78]:
import pandas as pd

insurance_data = pd.read_csv("../Med_insurance.csv")

### 2

Convert a categorical variable of your choice into dummy/indicator variables using the
pandas function pandas.get_dummies() and combine the result with the numerical columns
of the insurance_data


In [79]:
insurance = pd.get_dummies(insurance_data, columns=["smoker"], drop_first=True)

# now only keep the numeric columns
insurance = insurance.select_dtypes(include=["number"])

# print the first 5 rows of the data
insurance.head()

,age,bmi,children,charges,smoker_yes
0,19.0,27.900,0,16884.92400,1
1,18.0,33.770,1,1725.55230,0
2,28.0,33.000,3,4449.46200,0
3,33.0,22.705,0,21984.47061,0
4,32.0,28.880,0,3866.85520,0


### 3

Create a test set which is 20 % of the whole data set using a pure random sampling approach.

**Question**: Why do you need a test set when training a model?

**Answer**: To test the model on data it has not seen before.


In [80]:
from sklearn.model_selection import train_test_split
import random as rnd

train_set, test_set = train_test_split(
    insurance, test_size=0.2, random_state=rnd.seed(42)
)

# Print the number of rows in the train and test set
print("Number of rows in the train set: ", len(train_set))
print("Number of rows in the test set: ", len(test_set))

Number of rows in the train set:  1070
Number of rows in the test set:  268


## Exercise 2 - Preparing the data for Machine Learning algorithms

In the upcoming steps you will prepare the data that will be used to train a machine learning model

### 1

If you found missing values in the data add the missing entries for the respective column(s)
using the imputer transform (including only numerical
attributes) in Scikit SimpleImputer class. Use “median” as strategy. Make sure to train the imputer only on the training set.


In [81]:
# find the rows for missing data in the train set and print them and save the index
indexes = train_set[train_set.isnull().any(axis=1)].index
print(train_set.loc[indexes])

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

# fit the imputer to the train set
imputer.fit(train_set)

# transform the train set
X = imputer.transform(train_set)

# apply the imputer to the train set
train_set_fixed = pd.DataFrame(X, columns=train_set.columns, index=train_set.index)

print(train_set_fixed.loc[indexes])

# print the first 5 rows of the data
train_set_fixed.head()

      age    bmi  children     charges  smoker_yes
1332  NaN  44.70         3  11411.6850           0
1333  NaN  30.97         3  10600.5483           0
1330  NaN  25.74         2  12629.1656           0
1336  NaN  25.80         0   2007.9450           0
1335  NaN  36.85         0   1629.8335           0
1329  NaN  38.60         2  10325.2060           0
1337  NaN  29.07         0  29141.3603           1
1334  NaN  31.92         0   2205.9808           0
       age    bmi  children     charges  smoker_yes
1332  39.0  44.70       3.0  11411.6850         0.0
1333  39.0  30.97       3.0  10600.5483         0.0
1330  39.0  25.74       2.0  12629.1656         0.0
1336  39.0  25.80       0.0   2007.9450         0.0
1335  39.0  36.85       0.0   1629.8335         0.0
1329  39.0  38.60       2.0  10325.2060         0.0
1337  39.0  29.07       0.0  29141.3603         1.0
1334  39.0  31.92       0.0   2205.9808         0.0


,age,bmi,children,charges,smoker_yes
1213,52.0,33.3,2.0,10806.83900,0.0
867,57.0,43.7,1.0,11576.13000,0.0
732,24.0,30.1,3.0,4234.92700,0.0
34,28.0,36.4,1.0,51194.55914,1.0
1092,23.0,32.7,3.0,3591.48000,0.0


### b

We also need to interpalate the test data:

this is done completely seperate from the training data, so we can not use the same imputer.

```python

In [82]:
# find the rows for missing data in the test_set and print them and save the index
indexes = test_set[test_set.isnull().any(axis=1)].index
print(test_set.loc[indexes])

# fit the imputer to the test_set
imputer.fit(test_set)

# transform the test_set
X = imputer.transform(test_set)

# apply the imputer to the test_set
test_set_fixed = pd.DataFrame(X, columns=test_set.columns, index=test_set.index)

print(test_set_fixed.loc[indexes])

# print the first 5 rows of the data
test_set_fixed.head()

      age   bmi  children      charges  smoker_yes
1331  NaN  33.4         0  10795.93733           0
       age   bmi  children      charges  smoker_yes
1331  40.0  33.4       0.0  10795.93733         0.0


,age,bmi,children,charges,smoker_yes
853,53.0,23.750,2.0,11729.67950,0.0
178,46.0,28.900,2.0,8823.27900,0.0
984,20.0,30.115,5.0,4915.05985,0.0
1078,28.0,31.680,0.0,34672.14720,1.0
848,27.0,24.100,0.0,2974.12600,0.0


### 2

Perform feature scaling on all numerical attributes
using Scikit transform StandardScaler. Again, fit the scaler on the training data only.

***Question***: Explain what problem or problems the feature scaling resolves.

**Answer**: It makes the data more comparable, so that the model can learn better.

In [83]:
# Perform feature scaling on all numerical attributes
# using Scikit transform StandardScaler. Again, fit the scaler on the training data only.

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# fit the scaler to the train set
scaler.fit(train_set_fixed)

# transform the train set
X = scaler.transform(train_set_fixed)

# apply the scaler to the train set
train_set_scaled = pd.DataFrame(
    X, columns=train_set_fixed.columns, index=train_set_fixed.index
)

# print the first 5 rows of the data
train_set_scaled.head()

,age,bmi,children,charges,smoker_yes
1213,0.907695,0.460908,0.767250,-0.193662,-0.511657
867,1.262730,2.176427,-0.062839,-0.129207,-0.511657
732,-1.080501,-0.066944,1.597339,-0.744286,-0.511657
34,-0.796473,0.972265,-0.062839,3.190205,1.954436
1092,-1.151508,0.361936,1.597339,-0.798197,-0.511657


Save the prepocessed data into a variable `insurance_data_prepared`.


In [84]:
insurance_data_prepared = train_set_scaled

# Exercise 3 - Training a model

In the following steps you will select and train a machine learning model.

### 4

Now your data is ready to be used in training a machine learning model. Use it to train a Linear Regression model that can predict insurance charges. See the Training and Evaluation on the Training Set section at page 72 in the book [1].

**Question**: What is the difference between supervised and unsupervised learning? Is a Linear Regression supervised or unsupervised?

**Answer**: Supervised learning is when you have a dataset with the correct answers, and unsupervised learning is when you don't. Linear Regression is supervised.

**Question**: What is the difference between regression and classification?

**Answer**: Regression is when you want to predict a continuous value, and classification is when you want to predict a discrete value.


In [85]:
# Now your data is ready to be used in training a machine learning model. Use it to train a Linear Regression model that can predict insurance charges. See the Training and Evaluation on the Training Set section at page 72 in the book [1].

from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

# fit the model to the train set
lin_reg.fit(insurance_data_prepared, train_set["charges"])

# print the intercept and coefficients
print("Intercept: ", lin_reg.intercept_)
print("Coefficients: ", lin_reg.coef_)
print("Number of coefficients: ", len(lin_reg.coef_))


Intercept:  13118.263911915885
Coefficients:  [-3.30868365e-12 -1.37532447e-12 -1.08524747e-12  1.19353756e+04
 -5.74715538e-12]
Number of coefficients:  5


### 5

Test your trained model with the test data and find out the root mean squared error and
mean absolute error.


In [86]:
# Test your trained model with the test data and find out the root mean squared error and
# mean absolute error.